In [1]:
# load libraries
from quantplay.strategies.qplay_algorithm import QplayAlgorithm
from quantplay.utils.stock_utils import StockUtils
from quantplay.utils.alpha_factory import QuantplayAlphaFactory
from quantplay.utils.data_utils import DataUtils
import numpy as np
import pandas as pd

In [2]:
# load market data
day_candles = DataUtils.get_data(path='/NSE_EQ/', interval='day', stocks=["SBIN", "HDFCBANK"])
minute_candles = DataUtils.get_data(path='/NSE_EQ/', interval='minute', stocks=["SBIN", "HDFCBANK"])

time taken to load data for day candles --- 1.6947598457336426 seconds ---
time taken to load data for minute candles --- 8.083122730255127 seconds ---


In [3]:
# strategy class
class BTST(QplayAlgorithm):
    def __init__(self):
        super().__init__()

    def prepare_data(self):

        market = self.day_candle_data
        market.loc[:, "volume_5D_avg"] = (
            market.volume.rolling(5, min_periods=1).mean().shift(1)
        )
        market.loc[:, "volume_15D_avg"] = (
            market.volume.rolling(15, min_periods=1).mean().shift(1)
        )
        market.loc[
            :, "per_delivery"
        ] = market.deliverableQuantityToQuantityTradedRatio.shift(1)
        market.loc[:, "universe"] = market.is_top500.shift(1)
        market.loc[:, "prev_day_close"] = market.intraday_close.shift(1)
        market.loc[:, "above_sma"] = np.where(
            market.intraday_close.shift(1)
            > market.intraday_close.rolling(10).mean().shift(1),
            True,
            False,
        )
        market.loc[
            :, "annualized_volatility_shifted"
        ] = market.annualized_volatility.shift(1)

    def filter_trades(self):
        candle_data = self.candle_data
        market = self.day_candle_data

        QuantplayAlphaFactory.add_intraday_high_in_candle_data(candle_data)

        columns_to_remove = ["volume_5D_avg", "volume_15D_avg"]
        columns_to_remove = list(set(columns_to_remove) & set(candle_data.columns))
        candle_data = candle_data.drop(columns_to_remove, axis=1)

        candle_data = pd.merge(
            candle_data,
            market[["date_only", "symbol", "volume_5D_avg", "volume_15D_avg"]],
            how="left",
            left_on=["date_only", "symbol"],
            right_on=["date_only", "symbol"],
        )

        df = candle_data[
            (pd.to_datetime("13:00:00").time() <= candle_data.date.dt.time)
            & (pd.to_datetime("14:00:00").time() >= candle_data.date.dt.time)
            & (candle_data.total_volume > candle_data.volume_5D_avg)
            & (candle_data.total_volume > candle_data.volume_15D_avg)
        ].rename(columns={'close':'tick_close'})

        df_long = df.groupby(["date_only", "symbol"]).first().reset_index()

        df_long = df_long.rename(
            columns={
                "date": "trigger_order_tick_timstamp",
                "intraday_high": "trigger_price",
            }
        )

        columns_to_remove = ["trigger_order_tick_timstamp", "trigger_price"]
        columns_to_remove = list(set(columns_to_remove) & set(market.columns))
        market = market.drop(columns_to_remove, axis=1)

        trades_df = pd.merge(
            df_long,
            market,
            how="left",
            left_on=["date_only", "symbol"],
            right_on=["date_only", "symbol"],
        )

        # TODO P1 Keep on increasing value in pd filter
        trades_df = trades_df[trades_df.above_sma == True]
        trades_df = trades_df[trades_df.annualized_volatility_shifted >= 0.40]
        trades_df = trades_df[trades_df.universe == True]
        trades_df = trades_df[(trades_df.tick_close > trades_df.intraday_open) & (trades_df.tick_close > trades_df.prev_day_close)]
        trades_df = trades_df[trades_df.trigger_price > 0]

        return trades_df
    
    def get_trades(self):
        trades_df = super().get_trades()
        trades_df.loc[:, 'quantity'] = (trades_df.exposure/trades_df.trigger_price).astype(int)
        trades_df.loc[:, 'price'] = trades_df.trigger_price
        trades_df.loc[:, 'order_timestamp'] = trades_df.trigger_order_tick_timstamp
        
        return trades_df


In [4]:
minute_candles

,date,open,high,low,close,volume,symbol
0,2015-02-02 09:15:00,309.30,309.40,307.25,307.55,307430,SBIN
1,2015-02-02 09:16:00,307.85,308.00,307.50,307.65,105674,SBIN
2,2015-02-02 09:17:00,307.70,307.70,307.05,307.40,98199,SBIN
3,2015-02-02 09:18:00,307.45,308.35,307.35,308.25,114060,SBIN
4,2015-02-02 09:19:00,308.25,308.95,308.10,308.95,88416,SBIN
...,...,...,...,...,...,...,...
1303949,2022-02-18 15:25:00,1512.00,1514.00,1511.85,1513.95,18992,HDFCBANK
1303950,2022-02-18 15:26:00,1514.00,1514.00,1512.55,1512.90,7556,HDFCBANK
1303951,2022-02-18 15:27:00,1512.90,1513.25,1511.70,1513.10,15178,HDFCBANK
1303952,2022-02-18 15:28:00,1512.55,1513.00,1510.00,1510.05,7531,HDFCBANK
